In [ ]:
!python -m pip install lightning
!pip3 install torch torchvision torchaudio

!pip install tensorboard
!python -m pip install lightning
!pip3 install torch torchvision torchaudio
!pip install folium
!pip install selenium
!pip install hydra-core pytorch-lightning omegaconf torch torchvision torchaudio folium scipy matplotlib numpy pandas selenium
!pip install mlflow
!pip install kfp[kubernetes]
!pip install kserve
!pip install minio
!pip install nest_asyncio

In [ ]:
def setup_minio_stuff():
    import os
    from dotenv import load_dotenv
    
    import boto3

    load_dotenv()

    # # AWS access / secret keys are on the env file
    # minio_endpoint = os.getenv('AWS_ENDPOINT_URL_S3')

    # print(minio_endpoint)


    ################################################


    import os

    # Hardcoded AWS access and secret keys
    aws_access_key_id = 'minio'
    aws_secret_access_key = 'minio123'
    aws_endpoint_url_s3 = 'http://minio-service.kubeflow:9000'

    # Set the environment variables
    os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
    os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key
    os.environ['AWS_ENDPOINT_URL_S3'] = aws_endpoint_url_s3

    # Now you can access them as environment variables
    minio_endpoint = os.getenv('AWS_ENDPOINT_URL_S3')
    print(minio_endpoint)
    

    s3 = boto3.client('s3')
    return s3

def download_dir(prefix, local, bucket, client):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)





In [ ]:

import kfp
from kfp import dsl
from kfp.dsl import component, InputPath, OutputPath
from kfp import kubernetes

# In Jupyter Notebook
from main import main  # Import the main function from main.py

EXPERIMENT_NUMBER = 27


def train_model_op():
    import subprocess
    import os
    import boto3


    # from main import main
    # from hydra import compose, initialize
    # # Initialize Hydra and compose the configuration
    # initialize(config_path='conf')
    # cfg = compose(config_name='config')
    # main(cfg)
    
    def setup_minio_stuff():
        # # AWS access / secret keys are on the env file
        # minio_endpoint = os.getenv('AWS_ENDPOINT_URL_S3')

        # print(minio_endpoint)


        ################################################

        # Hardcoded AWS access and secret keys
        aws_access_key_id = 'minio'
        aws_secret_access_key = 'minio123'
        aws_endpoint_url_s3 = 'http://minio-service.kubeflow:9000'

        # Set the environment variables
        os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
        os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key
        os.environ['AWS_ENDPOINT_URL_S3'] = aws_endpoint_url_s3

        # Now you can access them as environment variables
        minio_endpoint = os.getenv('AWS_ENDPOINT_URL_S3')
        print(minio_endpoint)


        s3 = boto3.client('s3')
        return s3

    def download_dir(prefix, local, bucket, client):
        """
        params:
        - prefix: pattern to match in s3
        - local: local path to folder in which to place files
        - bucket: s3 bucket with target contents
        - client: initialized s3 client object
        """
        keys = []
        dirs = []
        next_token = ''
        base_kwargs = {
            'Bucket':bucket,
            'Prefix':prefix,
        }
        while next_token is not None:
            kwargs = base_kwargs.copy()
            if next_token != '':
                kwargs.update({'ContinuationToken': next_token})
            results = client.list_objects_v2(**kwargs)
            contents = results.get('Contents')
            for i in contents:
                k = i.get('Key')
                if k[-1] != '/':
                    keys.append(k)
                else:
                    dirs.append(k)
            next_token = results.get('NextContinuationToken')
        for d in dirs:
            dest_pathname = os.path.join(local, d)
            if not os.path.exists(os.path.dirname(dest_pathname)):
                os.makedirs(os.path.dirname(dest_pathname))
        for k in keys:
            dest_pathname = os.path.join(local, k)
            if not os.path.exists(os.path.dirname(dest_pathname)):
                os.makedirs(os.path.dirname(dest_pathname))
            client.download_file(bucket, k, dest_pathname)

    EXPERIMENT_STRING = 'experiment=experiment_fast_run_mil'
    s3 = setup_minio_stuff()
    download_dir('','/home/jovyan/dataset', 'dataset', s3)
    
    import time
    # time.sleep(600)
    
    
    subprocess.run(['python3', '/home/jovyan/flight_pattern_of_life/main.py', EXPERIMENT_STRING])
    
    # HYDRA_FULL_ERROR=1 /opt/anaconda3/envs/trajectory/bin/python /Users/aleksandranikevich/Desktop/AircraftTrajectory/REPO/flight_pattern_of_life/main.py experiment=experiment_fast_run_local

@dsl.pipeline(
    name=f'PyTorch Lightning Pipeline {EXPERIMENT_NUMBER}',
    description='A pipeline that trains a PyTorch Lightning model using Hydra'
)
def pytorch_lightning_pipeline():
    import subprocess
    
    train_op = train_model_op().set_display_name('Train Model').set_caching_options(False)
    train_op.set_env_variable('HYDRA_FULL_ERROR', '1')
    # ###volume_name = "aircraft-trajectory-volume"
    # volume_name = "dataset"
    
    # ###mount_path='/home/jovyan/aircraft-trajectory-volume'
    # mount_path = '/home/jovyan/dataset/ADIZ/Flightpath/'
    # kubernetes.mount_pvc(
    #     train_op,
    #     pvc_name=volume_name,
    #     mount_path=mount_path,    #'/data',
    # )
    
    
    # # mount_dir = "/mnt"
    
    # subprocess.run(['ls', '-l', mount_path])
    # #task = create_step_prepare_data().add_pvolumes({data_path: dsl.PipelineVolume(pvc=volume_name)})
    
    

# Compile the pipeline
pipeline_file = "pytorch-lightning-pipeline.yaml"
with open(pipeline_file, "w") as f:
    kfp.compiler.Compiler().compile(
        pipeline_func=pytorch_lightning_pipeline,
        package_path=f.name
    )

# Experiment setup and run
EXPERIMENT_NAME = 'AA test pipeline'
from kfp.client.set_volume_credentials import ServiceAccountTokenVolumeCredentials

namespace = "kubeflow"
credentials = ServiceAccountTokenVolumeCredentials(path='/var/run/secrets/kubeflow/pipelines/token')
client = kfp.Client(host=f"http://ml-pipeline-ui.{namespace}", credentials=credentials)

client.create_run_from_pipeline_func(
    pytorch_lightning_pipeline,
    arguments={},
    experiment_name=EXPERIMENT_NAME
)

pipeline = client.upload_pipeline(
    pipeline_package_path=pipeline_file,
    pipeline_name=f"PytorchLightningPipeline {EXPERIMENT_NUMBER}",
    description=f"Test pipeline pytorch lightning {EXPERIMENT_NUMBER}"
)

# pipeline_name = "PytorchLightningPipeline 35"
# pipeline = client.upload_pipeline_version(
#     pipeline_package_path=pipeline_file,
#     pipeline_name=pipeline_name,
#     pipeline_version_name="0.0.1",
#     description="some pipeline"
# )




In [1]:

import kfp
from kfp import dsl
from kfp.dsl import component, InputPath, OutputPath
from kfp import kubernetes

# In Jupyter Notebook
from main import main  # Import the main function from main.py

EXPERIMENT_NUMBER = 1
EXPERIMENT_STRING = 'experiment=experiment_fast_run_mil'



@component(
    #base_image="ghcr.io/raft-tech/raft-kubeflow:yolo-train-v1",
    base_image="ghcr.io/raft-tech/aircraft_trajectory:0.1.0",
    ##packages_to_install=['mlflow', 'pytorch-lightning']
)
def train_model_op():
    import subprocess


    # from main import main
    # from hydra import compose, initialize
    # # Initialize Hydra and compose the configuration
    # initialize(config_path='conf')
    # cfg = compose(config_name='config')
    # main(cfg)
    
    
    
    
    subprocess.run(['python3', '/home/jovyan/flight_pattern_of_life/main.py', EXPERIMENT_STRING])
    
    # HYDRA_FULL_ERROR=1 /opt/anaconda3/envs/trajectory/bin/python /Users/aleksandranikevich/Desktop/AircraftTrajectory/REPO/flight_pattern_of_life/main.py experiment=experiment_fast_run_local

@dsl.pipeline(
    name=f'PyTorch Lightning Pipeline {EXPERIMENT_NUMBER}',
    description='A pipeline that trains a PyTorch Lightning model using Hydra'
)
def pytorch_lightning_pipeline():
    import subprocess
    
    train_op = train_model_op().set_display_name('Train Model').set_caching_options(False)
    train_op.set_env_variable('HYDRA_FULL_ERROR', '1')
    volume_name = "aircraft-trajectory-volume"
    ######train_op.add_pvolumes({'data_path': PipelineVolume(pvc=volume_name)})
    
    mount_path='/home/jovyan/aircraft-trajectory-volume'
    kubernetes.mount_pvc(
        train_op,
        pvc_name=volume_name,
        mount_path=mount_path,    #'/data',
    )
    
    
    # mount_dir = "/mnt"
    
    subprocess.run(['ls', '-l', mount_path])
    #task = create_step_prepare_data().add_pvolumes({data_path: dsl.PipelineVolume(pvc=volume_name)})
    
    

# Compile the pipeline
pipeline_file = "pytorch-lightning-pipeline.yaml"
with open(pipeline_file, "w") as f:
    kfp.compiler.Compiler().compile(
        pipeline_func=pytorch_lightning_pipeline,
        package_path=f.name
    )

# Experiment setup and run
EXPERIMENT_NAME = 'AA test pipeline'
from kfp.client.set_volume_credentials import ServiceAccountTokenVolumeCredentials

namespace = "kubeflow"
credentials = ServiceAccountTokenVolumeCredentials(path='/var/run/secrets/kubeflow/pipelines/token')
client = kfp.Client(host=f"http://ml-pipeline-ui.{namespace}", credentials=credentials)

client.create_run_from_pipeline_func(
    pytorch_lightning_pipeline,
    arguments={},
    experiment_name=EXPERIMENT_NAME
)

pipeline = client.upload_pipeline(
    pipeline_package_path=pipeline_file,
    pipeline_name=f"PytorchLightningPipeline {EXPERIMENT_NUMBER}",
    description=f"Test pipeline pytorch lightning {EXPERIMENT_NUMBER}"
)

# pipeline_name = "PytorchLightningPipeline 35"
# pipeline = client.upload_pipeline_version(
#     pipeline_package_path=pipeline_file,
#     pipeline_name=pipeline_name,
#     pipeline_version_name="0.0.1",
#     description="some pipeline"
# )




ModuleNotFoundError: No module named 'kfp'

In [ ]:
import base64
import io
import os
from typing import Dict

import kserve
import requests
import torch
from dotenv import load_dotenv
from minio import Minio
from minio.error import S3Error



class AircraftTrajectoryPredictor(kserve.Model):
    def __init__(self, name: str, model_file: str):
        super().__init__(name)
        self.name = name
        self.model_file = model_file
        self.model = None
        self.ready = False
        self.minio_client = Minio(
            os.environ.get("MINIO_ENDPOINT", "minio-service.kubeflow:9000"),
            access_key="minio", #os.environ.get("MINIO_ACCESS_KEY"),
            secret_key="minio123", #os.environ.get("MINIO_SECRET_KEY"),
            secure=False,  # Set to True if using HTTPS
        )
        self.load()

    def load(self):
        try:
            # First, check if the model file exists locally
            if os.path.exists(self.model_file):
                model_path = self.model_file
            else:
                # If not local, check if the model file exists in the MinIO bucket
                bucket_name = os.environ.get("MINIO_BUCKET", "aanikevich")
                self.minio_client.stat_object(bucket_name, self.model_file)

                # If the file exists in MinIO, download it to a temporary location
                model_path = f"/tmp/{self.model_file}"
                self.minio_client.fget_object(bucket_name, self.model_file, model_path)

            # Load the model using the file path
                self.model = torch.jit.load(model_path)
            # Clean up the temporary file if it was downloaded from MinIO
            if model_path.startswith("/tmp/"):
                os.remove(model_path)
            
            # Model ready
            self.ready = True
        except S3Error as e:
            raise RuntimeError(f"Error accessing model file in MinIO: {str(e)}")
        except FileNotFoundError:
            raise RuntimeError(f"Model file not found: {self.model_file}")
        except Exception as e:
            raise RuntimeError(f"Error loading model: {str(e)}")

    def predict(self, request: Dict, headers: Dict) -> Dict:
        # Implement prediction logic here
        if not self.ready:
            raise RuntimeError("Model is not loaded")

        # Process input and make predictions
        input_data = request["instances"]
        # Raw input data
        print(input_data)

        out_loaded = self.model(input_data)
        
        results = out_loaded.detach().cpu().numpy().tolist()

        # Return the JSON response
        return {"predictions": results}
    
    


In [2]:
import nest_asyncio
import asyncio
import kserve

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()


model_file = "models_jit2/scripted_model.pt"
model_name = "aircraft-trajectory"
predictor = AircraftTrajectoryPredictor(model_name, model_file)
predictor.load()

# Start the model server without running into the nested event loop issue
kserve.ModelServer().start([predictor]) 

NameError: name 'AircraftTrajectoryPredictor' is not defined

In [3]:

example_input = torch.rand(3, 5, 100)
example_dict = {"instances": example_input}

predictor.predict(example_dict)

NameError: name 'torch' is not defined